In [1]:
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils
from keras import optimizers
from keras import losses

DATA_PATH = './data/train/'
TEST_PERCENT = 0.1
SELECT_SUBSET_PERCENT = 0.3

# The cat and dog images are of variable size we have to resize them to all the same size.
RESIZE_WIDTH=32
RESIZE_HEIGHT=32
EPOCHS = 10

Using TensorFlow backend.


In [9]:
import matplotlib
from matplotlib import pyplot

X = []
Y = []

files = os.listdir(DATA_PATH)
# Shuffle so we are selecting about an equal number of dog and cat images.
shuffled_files = sklearn.utils.shuffle(files)
select_count = int(len(shuffled_files) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count)

subset_files_select = shuffled_files[:select_count]

DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = matplotlib.pyplot.imread(DATA_PATH + input_file)
    # Resize the images to be the same size.
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    if 'cat' == input_file.split('.')[0]:
        Y.append(0.0)
    else:
        Y.append(1.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255

Going to load 7500 files
Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Have loaded 4000 samples
Have loaded 5000 samples
Have loaded 6000 samples
Have loaded 7000 samples
Train set has dimensionality (6750, 32, 32, 3)
Test set has dimensionality (750, 32, 32, 3)


In [10]:
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(RESIZE_WIDTH, RESIZE_HEIGHT, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))

# Define loss and objective
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
batch_size = 150

model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 5400 samples, validate on 1350 samples
Epoch 1/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5384 - acc: 0.7333 - val_loss: 0.5501 - val_acc: 0.7126
Epoch 2/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5343 - acc: 0.7393 - val_loss: 0.5881 - val_acc: 0.6733
Epoch 3/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5290 - acc: 0.7367 - val_loss: 0.5365 - val_acc: 0.7304
Epoch 4/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5240 - acc: 0.7406 - val_loss: 0.5559 - val_acc: 0.7037
Epoch 5/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5196 - acc: 0.7394 - val_loss: 0.5403 - val_acc: 0.7148
Epoch 6/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5102 - acc: 0.7539 - val_loss: 0.5376 - val_acc: 0.7237
Epoch 7/10
5400/5400 [==============================] - 18s 3ms/step - loss: 0.5095 - acc: 0.7513 - val_loss: 0.5295 - val_acc: 0.7348
Epoch 8

In [ ]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))